In [ ]:
import pip
pip.main(["install", "openpyxl"])

In [4]:
import pandas as pd
# import openpyxl

In [58]:
df = pd.read_excel("../../data/production/WMD2022 Commodities 1984-2020.xlsx")
df

,Minerals in metric tons*,1984,1985,1986,1987,1988,1989,1990,1991,1992,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,Es inicio
0,Iron Ore (Fe-cont.),446887838.0,492697409.0,495584899.0,507784666.0,524316505.0,539637227.0,529023870.0,531831265.0,499087427.0,...,1.281792e+09,1.359375e+09,1.448968e+09,1.456311e+09,1.466177e+09,1.511759e+09,1.523930e+09,1.552366e+09,1.522559e+09,1.0
1,Albania,439502.0,414035.0,379154.0,444690.0,488050.0,539314.0,425697.0,203830.0,10000.0,...,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Algeria,2000000.0,1836000.0,1836000.0,1836000.0,1674000.0,1483920.0,1588140.0,1266000.0,1362400.0,...,8.424000e+05,5.940000e+05,4.920100e+05,5.098000e+05,3.283000e+05,2.684000e+05,3.812000e+05,4.099000e+05,4.066000e+05,NaN
3,Argentina,345576.0,388567.0,514259.0,359902.0,378576.0,414425.0,444115.0,88932.0,2502.0,...,3.324700e+05,2.981900e+05,2.248400e+05,2.060600e+05,1.072800e+05,2.840000e+04,4.056000e+04,0.000000e+00,0.000000e+00,NaN
4,Australia,56885000.0,62042117.0,60081983.0,64798343.0,61058000.0,66660300.0,66402630.0,74133990.0,72075000.0,...,3.219525e+08,3.777600e+08,4.574092e+08,5.009936e+08,5.310754e+08,5.482971e+08,5.600546e+08,5.679571e+08,5.683846e+08,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2661,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2662,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2663,* Minerals in metric tons except,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2664,"Rhenium, Precious Metals in kg",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Minerals in metric tons except
### Rhenium, Precious Metals in kg
### Natural Gas in Million m3


In [59]:
# Get headers
header_indexes = df.index[df["Es inicio"] == 1.0].tolist()
# Get rid of "Es inicio" column
del df["Es inicio"]

def convert_df(ugly_df):
    # We get the name of the material from first column and first row
    mineral_name = ugly_df.iloc[0, 0]
    # Remove the 1st cell since it's a header
    tmp_df = ugly_df.iloc[1:]
    # Add column with the name of mineral/thing
    tmp_df["Product"] = mineral_name
    # Change the 1st column as Country
    tmp_df.rename(columns={"Minerals in metric tons*": "Country"}, inplace=True)
    # Melt the df into a conventional table    
    melted_df = tmp_df.melt(
        id_vars=["Country", "Product"],
        var_name="Year",
        value_name="Value"
    )
    return melted_df

list_df = list()
for n in range(len(header_indexes) - 1):
    # print(header_indexes[n], header_indexes[n + 1])
    tmp_df = df.iloc[header_indexes[n]:header_indexes[n + 1]]
    melted_df = convert_df(tmp_df)
    list_df.append(melted_df)
# and the last one. Skip last 5 rows because of the annotations
tmp_df = df.iloc[header_indexes[-1]:df.shape[0] - 5]
melted_df = convert_df(tmp_df)
list_df.append(melted_df)


c:\Users\zacat\OneDrive\Documentos\Investigacion\investigacion-gnn-cic\rb_venv3715\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == "":
c:\Users\zacat\OneDrive\Documentos\Investigacion\investigacion-gnn-cic\rb_venv3715\lib\site-packages\pandas\core\frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [60]:
list_df[-1]

,Country,Product,Year,Value
0,Argentina,Uranium (U3O8-cont.),1984,151.0
1,Australia,Uranium (U3O8-cont.),1984,5099.0
2,Belgium,Uranium (U3O8-cont.),1984,59.0
3,Brazil,Uranium (U3O8-cont.),1984,138.0
4,Bulgaria,Uranium (U3O8-cont.),1984,0.0
...,...,...,...,...
1179,Spain,Uranium (U3O8-cont.),2020,NaN
1180,Ukraine,Uranium (U3O8-cont.),2020,877.0
1181,United States,Uranium (U3O8-cont.),2020,7.0
1182,Uzbekistan,Uranium (U3O8-cont.),2020,4127.0
